## Modelagem IMDB

Explique como você faria a previsão da nota do imdb a partir dos dados. Quais variáveis e/ou suas transformações você utilizou e por quê? Qual tipo de problema estamos resolvendo (regressão, classificação)? Qual modelo melhor se aproxima dos dados e quais seus prós e contras? Qual medida de performance do modelo foi escolhida e por quê?


##### Previsão da Nota IMDb
Para prever a nota do IMDb, vamos usar um modelo de regressão. A tarefa é um problema de regressão porque estamos prevendo um valor contínuo.

##### Seleção de Variáveis e Transformações
Variáveis utilizadas:

- Released_Year
- Runtime
- Meta_score
- No_of_Votes
- Gross

Transformações:

Conversão de Gross para valores numéricos
Codificação de variáveis categóricas (por exemplo, Genre)

##### Modelo de Regressão
Vamos usar um modelo de regressão linear para prever a nota do IMDb.

In [14]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings("ignore")
import joblib

In [15]:
# Carregar o DataFrame a partir de um arquivo pickle
df = pd.read_pickle('df.pkl')
df.head(3)

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
1,The Godfather,1972.0,A,175,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,134966411.0
2,The Dark Knight,2008.0,UA,152,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,534858444.0
3,The Godfather: Part II,1974.0,A,202,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,57300000.0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 998 entries, 1 to 999
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Series_Title   998 non-null    object 
 1   Released_Year  998 non-null    float64
 2   Certificate    897 non-null    object 
 3   Runtime        998 non-null    int64  
 4   Genre          998 non-null    object 
 5   IMDB_Rating    998 non-null    float64
 6   Overview       998 non-null    object 
 7   Meta_score     998 non-null    float64
 8   Director       998 non-null    object 
 9   Star1          998 non-null    object 
 10  Star2          998 non-null    object 
 11  Star3          998 non-null    object 
 12  Star4          998 non-null    object 
 13  No_of_Votes    998 non-null    int64  
 14  Gross          998 non-null    float64
dtypes: float64(4), int64(2), object(9)
memory usage: 124.8+ KB


In [28]:
# Separar as features e o target
X = df.drop('IMDB_Rating', axis=1)
y = df['IMDB_Rating']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [29]:
# Preprocessamento
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('normalização', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('numérica', numeric_transformer, numeric_features),
        ('categórica', categorical_transformer, categorical_features)])

In [30]:
# Modelos base
models = {
    'LR': LinearRegression(),
    'DT': DecisionTreeRegressor(),
    'RF': RandomForestRegressor(),
    'GB': GradientBoostingRegressor(),
    'MLP': MLPRegressor(),
    'SVR': SVR()
}

In [31]:
# Definir os parâmetros para RandomizedSearchCV
param_distributions = {
    'LR': {},
    'DT': {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 10, 20]},
    'RF': {'n_estimators': [50, 100, 200], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, None], 'min_samples_split': [2, 5, 10]},
    'GB': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 4, 5]},
    'MLP': {'hidden_layer_sizes': [(50,50), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']},
    'SVR': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
}

# Otimização dos hiperparâmetros e treinamento
best_estimators = {}
for name, model in models.items():
    search = RandomizedSearchCV(model, param_distributions[name], n_iter=10, scoring='neg_mean_squared_error', cv=5, random_state=42)
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), (name, search)])
    pipeline.fit(X_train, y_train)
    best_estimators[name] = pipeline.named_steps[name].best_estimator_

In [32]:
# Criar o Voting Regressor com os melhores estimadores
voting_regressor = VotingRegressor(estimators=[
    ('LR', best_estimators['LR']),
    ('DT', best_estimators['DT']),
    ('RF', best_estimators['RF']),
    ('GB', best_estimators['GB']),
    ('MLP', best_estimators['MLP']),
    ('SVR', best_estimators['SVR'])
])

# Pipeline final
final_pipeline = Pipeline(steps=[('Pré-Processamento', preprocessor), ('Ensemble Voting', voting_regressor)])
final_pipeline.fit(X_train, y_train)

Pipeline(steps=[('Pré-Processamento',
                 ColumnTransformer(transformers=[('numérica',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('normalização',
                                                                   StandardScaler())]),
                                                  Index(['Released_Year', 'Runtime', 'Meta_score', 'No_of_Votes', 'Gross'], dtype='object')),
                                                 ('categórica',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncod...
                 VotingRegressor(estimators=[('LR', LinearRegression()),
                                             ('DT',
                                              DecisionTreeRegressor(max_depth=10,
                                                                    min_samples_split=20)),
                                             ('RF',
                                              RandomForestRegressor(max_depth=30,
                                                                    max_features='sqrt',
                                                                    min_samples_split=5,
                                                                    n_estimators=200)),
                                             ('GB',
                                              GradientBoostingRegressor(n_estimators=200)),
                                             ('MLP',
                                              MLPRegressor(hidden_layer_sizes=(50,
                                                                               50),
                                                           solver='sgd')),
                                             ('SVR', SVR(C=1))]))])

In [33]:
# Avaliação do modelo construído
y_pred = final_pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

MAE: 0.15045931048584305
MSE: 0.037956405541239795
RMSE: 0.1948240373805034
R^2: 0.4233981027569712


In [35]:
# Salvar o modelo
#joblib.dump(final_pipeline, 'avaliacao_imdb_model.pkl')

['avaliacao_imdb_model.pkl']

### Predição da Avaliação IMDb do Filme 'The Shawshank Redemption'

In [5]:
# Características do filme
new_movie = {'Released_Year': 1994, 'Runtime': 142, 'Meta_score': 80.0, 'No_of_Votes': 2343110, 'Gross': 28341469}

# Criar um DataFrame
new_movie_df = pd.DataFrame([new_movie])

# Previsão da nota IMDb
predicted_rating = model.predict(new_movie_df)
print(f'Nota IMDb prevista: {round(predicted_rating[0],2)}')

Nota IMDb prevista: 9.21


### Apêndice

In [4]:
'''# Seleção de variáveis
X = df[['Released_Year', 'Runtime', 'Meta_score', 'No_of_Votes', 'Gross']]
y = df['IMDB_Rating']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error: {rmse}')'''

Root Mean Squared Error: 0.21075340555277375
